In [ ]:

!pip3 install transformers optimum
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

!pip install -qU \
sentence-transformers==2.2.2 \
accelerate==0.21.0 \
einops==0.6.1 \
langchain \
#xformers==0.0.20 \



Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-gptq 0.5.1+cu118 requires accelerate>=0.22.0, but you have accelerate 0.21.0 which is incompatible.


In [ ]:
!pip install "weaviate-client==3.*"
!pip install rank_bm25

In [ ]:
! pip install langchain auto-gptq streamlit pyngrok

  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [ ]:
%%writefile app.py
import streamlit as st
import transformers
import streamlit as st
from torch import cuda, bfloat16
from langchain.vectorstores import Weaviate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
import weaviate
import sys
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from auto_gptq import exllama_set_max_input_length
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.schema.prompt_template import BasePromptTemplate

def logout():
    st.balloons()
    st.warning("Logged out successfully!")
    st.session_state.logged_in = False

# Initialize session state
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
@st.cache_resource
def pipeln():
    device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
    model_name_or_path= "TheBloke/zephyr-7B-beta-GPTQ"

    model_name = "BAAI/bge-base-en-v1.5"
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

    embedding_function = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cuda'},
        encode_kwargs=encode_kwargs,

    )

    loader = CSVLoader(file_path="/content/cleaned_data (3).csv")
    documents = loader.load()
    WEAVIATE_URL = "https://hackaphasia-data-idfb9vuc.weaviate.network"
    WEAVIATE_API_KEY = "ssznXL9zZv68HnjZJ66ZoIhhBUvyIcgTG3R2"

    weaviate_client = weaviate.Client(
        url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
    )
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    embedding = embedding_function
    weaviate_vectorstore = Weaviate.from_documents(texts, embedding, client=weaviate_client, by_text=False)
    weaviate_retriever = weaviate_vectorstore.as_retriever(search_kwargs={"k":10})





    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                device_map="auto",
                                                trust_remote_code=False,
                                                revision="main")
    model = exllama_set_max_input_length(model, max_input_length=7000)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    generate_text = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.5,
    repetition_penalty=1.1,
    top_p=0.95,
    top_k=1,

    )
    llm = HuggingFacePipeline(pipeline=generate_text)
    prompt_template = """Use the following pieces of context to answer the question at the end accuretly. If you don't know the answer, just say that you don't know, don't try to make up an answer and
                     If context and question doesn't match do not give answer, just say that I don't know and say that it is not related to
                     the patent data even if is anything outside of the patent data then say you can only answer on patent data and not anythin else.
    {context}

    Question: {question}
    Answer :"""
    CONV_PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    chat = ConversationalRetrievalChain.from_llm(llm, retriever=weaviate_retriever, memory=memory,combine_docs_chain_kwargs={"prompt":CONV_PROMPT})
    return chat,weaviate_retriever

if not st.session_state.logged_in:
    st.subheader("Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    login_button = st.button("Login")

    if login_button:
        # Use a secure authentication method here
        # ...

        if username == "User" and password == "user@123":
            st.session_state.logged_in = True
            st.success("Login successful!")

if st.session_state.logged_in:


# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"

    def main():

        menu = ['Home', 'About', 'Chat']
        ch = st.sidebar.selectbox('Menu', menu)
        if ch == 'Home':
            st.markdown("<h1 style='text-align: center;'>-Welcome to <span style='color: lightblue;'>SciBot🤖-</span></h1>", unsafe_allow_html=True)
            st.markdown("<h4>Specific Requirements: </h4>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Welcome to SciBot!</span></div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Get all your queries related to patent resolved in minutes </span></div>", unsafe_allow_html=True)
            st.subheader('Home page')
        if ch == 'About':
            st.markdown("<h1 style='text-align: center;'>-ABOUT-</h1>", unsafe_allow_html=True)
            st.markdown("<div style='text-align: center;'><span style='font-size: 22px; font-weight: 700;'>Problem Statement: </span>Chatbot to explore science and technologies</h3>", unsafe_allow_html=True)
            st.markdown("<div style='text-align: justify; margin-top: 2vh;'>Traditional keyword-based patent searches face limitations with complex jargon and lengthy documents, hindering semantic understanding. To overcome this, advanced information retrieval systems leverage Natural Language Processing and machine learning. Semantic search algorithms analyze word relationships, enhancing relevance. Machine learning models, like deep learning architectures, categorize, summarize, and extract key patent information from extensive texts. User interfaces with natural language capabilities improve user experience, allowing intuitive queries. Overall, integrating NLP, machine learning, and user-friendly interfaces addresses keyword search limitations, promising more efficient and accurate patent information retrieval.</div>", unsafe_allow_html=True)
            st.markdown("<h4>Specific Requirements: </h4>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Semantic Understanding: </span>The chatbot must go beyond keyword matching, understanding the semantic context of user queries.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Information Extraction: </span>Capable of scanning through patent text and extracting relevant information in response to the queries.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Interactivity: </span>Engage with users in a conversational manner, allowing for follow-up questions and clarification.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Summarization: </span>Provide concise summaries of extracted information, maintaining accuracy and relevance.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>User Query Learning: </span>Learn from user interactions to improve response accuracy over time.</div>", unsafe_allow_html=True)
            st.markdown("<div><span style='font-weight: 700;'>Scalability: </span>Efficiently handle a large volume of articles and simultaneous user queries.</div>", unsafe_allow_html=True)

        if ch == 'Chat':
            st.title("Chat Page")

            if "messages" not in st.session_state:
                st.session_state.messages = []

            for message in st.session_state.messages:
                with st.chat_message(message["role"]):
                    st.markdown(message["content"])

            if prompt := st.chat_input("Enter text: "):
                with st.chat_message("user"):
                    st.markdown(prompt)
                st.session_state.messages.append({"role" :"user", "content":prompt})
                chat,weaviate_retriever = pipeln()
                ans = ""
                docs = weaviate_retriever.get_relevant_documents(prompt)
                for document in docs:
                    ans += f"{document.metadata.get('title')} {document.metadata.get('URL')} {document.page_content}"+"\n"
                ans = ans.replace("None None", "").strip()
                answer_for_retreive = ans

                bot_response = chat({"question": prompt})['answer'] +"\n"+ answer_for_retreive
                response=f"SciBot: {bot_response}"
                with st. chat_message("assistant"):
                    st.markdown(response)
                st.session_state.messages.append({"role": "assistant", "content": response})


    if __name__ == '__main__':
        main()

    # Implement chatbot functionality here using NLP libraries

if st.sidebar.button("Logout"):
  logout()



Overwriting app.py


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
!pip install pyngrok==4.1.1
from pyngrok import ngrok
# !ngrok config add-authtoken 2ZVwALoImirOuZfwbb6dws6KO0G_829DDbQD1RL1TPbr5b1sA


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!ngrok authtoken 2ZWznGkzV6spslw5MwUFZOV9vSg_6uNCaqes8eiuUV7EHN8ab
#!ngrok authtoken NGROK_Token
public_url = ngrok.connect(port='8501')
print(public_url)

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
http://d5ba-35-233-148-190.ngrok-free.app
